<a href="https://colab.research.google.com/github/ssirin/EnergyTransition/blob/ColabCode/ClimateBert_Energy_Text_Analysis_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [7]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00


In [8]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [28]:
## Create a function to analyze all data and return data frame
def energy_text_analysis(dataframe):
  df = dataframe.copy()
  # Define KeyDataset class to extract a specific field from the data
  from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
  from transformers.pipelines.pt_utils import KeyDataset
  import datasets
  from tqdm.auto import tqdm
  import pandas as pd
  from torch.utils.data import Dataset

  class KeyDataset(Dataset):
      def __init__(self, df, key):
          self.df = df
          self.key = key

      def __len__(self):
          return len(self.df)

      def __getitem__(self, idx):
          return {self.key: self.df.loc[idx, self.key]}

## Analyze text specificity

  model_name = "climatebert/distilroberta-base-climate-specificity"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)
  pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
  climate_specificity = []

   # Iterate through DataFrame rows and apply pipeline
  for idx, row in tqdm(df.iterrows(), total=df.shape[0]):

      value  = row['ClimateText']
      if value == 0 :
        out = "None"
        climate_specificity.append(out)
      else:
        # Extract 'Risk Factors' text and put it in a DataFrame
        temp_df = pd.DataFrame({'text': [row['Risk Factors']]})

        # Apply the pipeline
        for out in pipe(KeyDataset(temp_df, 'text'), padding=True, truncation=True):
            # Append results to list (modify as needed based on actual output)
            climate_specificity.append(out)

  # Add results to new DataFrame column
  df['ClimateSpecficity'] = climate_specificity

  ###########################################################################################

  ## Analysis for TCDF

  model_name = "climatebert/distilroberta-base-climate-tcfd"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)
  pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

  # Initialize empty list to store results
  tcfd_results = []

  # Iterate through DataFrame rows and apply pipeline
  for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
     value  = row['ClimateText']
     if value == 0 :
        out = "None"
        tcfd_results.append(out)
     else:
        # Extract 'Risk Factors' text and put it in a DataFrame
        temp_df = pd.DataFrame({'text': [row['Risk Factors']]})

        # Apply the pipeline
        for out in pipe(KeyDataset(temp_df, 'text'), padding=True, truncation=True):
          # Append results to list (modify as needed based on actual output)
          tcfd_results.append(out)

  # Add results to new DataFrame column
  df['TCFD'] = tcfd_results


##################################################################################
## Analysis for Transition-Pyhsical


  model_name = "climatebert/transition-physical"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer_name = "climatebert/distilroberta-base-climate-detector"
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, max_len=512)
  pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

  # Initialize empty list to store results
  transition_results = []

  # Iterate through DataFrame rows and apply pipeline
  for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    value  = row['ClimateText']
    if value == 0 :
      out = "None"
      transition_results.append(out)
    else:
      # Extract 'Risk Factors' text and put it in a DataFrame
      temp_df = pd.DataFrame({'text': [row['Risk Factors']]})

      # Apply the pipeline
      for out in pipe(KeyDataset(temp_df, 'text'), padding=True, truncation=True):
          # Append results to list (modify as needed based on actual output)
          transition_results.append(out)

  # Add results to new DataFrame column
  df['Transition'] = transition_results

########################################################################################

  ## Apply climate sentiment

  model_name = "climatebert/distilroberta-base-climate-sentiment"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)
  pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
  # Initialize empty list to store results
  sentiment_results = []

  # Iterate through DataFrame rows and apply pipeline
  for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    value  = row['ClimateText']
    if value == 0 :
      out = "None"
      sentiment_results.append(out)
    else:
      # Extract 'Risk Factors' text and put it in a DataFrame
      temp_df = pd.DataFrame({'text': [row['Risk Factors']]})
      # Apply the pipeline
      for out in pipe(KeyDataset(temp_df, 'text'), padding=True, truncation=True):
          # Append results to list (modify as needed based on actual output)
          sentiment_results.append(out)

  # Add results to new DataFrame column
  df['Sentiment'] = sentiment_results


#############################################################################
  ## Apply renewable energy results


  model_name = "climatebert/renewable"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer_name = "climatebert/distilroberta-base-climate-detector"
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, max_len=512)
  pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

  ## Initialize empty list to store results
  renewable_results = []

  # Iterate through DataFrame rows and apply pipeline
  for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    value  = row['ClimateText']
    if value == 0 :
      out = "None"
      renewable_results.append(out)
    else:
      # Extract 'Risk Factors' text and put it in a DataFrame
      temp_df = pd.DataFrame({'text': [row['Risk Factors']]})

      # Apply the pipeline
      for out in pipe(KeyDataset(temp_df, 'text'), padding=True, truncation=True):
          # Append results to list (modify as needed based on actual output)
          renewable_results.append(out)

  # Add results to new DataFrame column
  df['Renewable'] = renewable_results


  return(df)


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/SEC_BERT_Results/sec_bert_2005.csv')
new_df = energy_text_analysis(df)
new_df.to_csv('/content/drive/MyDrive/SEC_BERT_Results/sec_bert_corrected_2005.csv', index = False)

  0%|          | 0/12290 [00:00<?, ?it/s]

  0%|          | 0/12290 [00:00<?, ?it/s]

  0%|          | 0/12290 [00:00<?, ?it/s]

  0%|          | 0/12290 [00:00<?, ?it/s]

  0%|          | 0/12290 [00:00<?, ?it/s]